# Plotando com Makie 

Existem diversos pacotes para plotagem com Julia (e.g. Plots.jl, VegaLite.jl, Gadfly.jl,...), entretanto,
o pacote Makie vem ganhando destaque. Na comunidade de Julia, é comum ouvir que "Makie is the future!".
As razões são diversas, entre elas está o fato de o pacote ser totalmente nativo em Julia, simples iteratividade,
bom sistema de layout, entre outros. Mas o que me convenceu (e possivelmente outros da comunidade),
foram alguns dos belíssimos exmemplos de plots como os presentes no repositório [BeautifulMakie](https://lazarusa.github.io/BeautifulMakie/).

Além disso, muitos pacotes interessantes tem sido construídos em cima do Makie, como o [AlgebraOfGraphics.jl](https://github.com/JuliaPlots/AlgebraOfGraphics.jl).

Entretanto, é preciso falar alguns dos atuais "shortcomings" do Makie. O primeiro é algo bastante
comum para quem usa Julia. O primeiro plot leva um tempo considerável devido ao tempo de pré-compilção (felizmente,
isso pode ser contornado com o PkgCompiler.jl, olhe esse meu [artigo aqui](https://medium.com/coffee-in-a-klein-bottle/speeding-up-julia-precompilation-97f39d151a9f)). O segundo problema é que o pacote ainda está em franco desenvolvimento, e não está tão maduro
como o Plots.jl. Assim, não é incomum encontrar bugs de vez em quando, ou descobrir que uma feature
específica ainda não foi implementada.

Mesmo assim, esses são somente percalços frente ao que o Makie pode oferecer.

*obs: sugiro fortemente seguir o tutorial [aqui](https://medium.com/coffee-in-a-klein-bottle/speeding-up-julia-precompilation-97f39d151a9f) para
agilizar a pré-compilação.*

O primeiro ponto não convencional ao se usar o Makie é o fato que ele permimte o uso de diferentes
backends para plotagem. Você pode olhar na documentação, mas vamos falar de dois especificamente, eles são o CairoMakie
e o GLMakie. O CairoMakie produz imagens estáticas e de alta qualidade (e.g. svg), sendo assim ideal para usar
quando se quer produzir um imagem para publicação. Já o GLMakie 

In [16]:
using CairoMakie
CairoMakie.activate!(type = "svg")
using AlgebraOfGraphics
kwargs = AlgebraOfGraphics.aog_theme()
set_theme!(;resolution=(500,500),kwargs...)

In [19]:
f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98),
    resolution = (1000, 700));

In [46]:
ga = f[1, 1] = GridLayout()
gb = f[2, 1] = GridLayout()
gcd = f[1:2, 2] = GridLayout()
gc = gcd[1, 1] = GridLayout()
gd = gcd[2, 1] = GridLayout()

GridLayout[1, 1] with 0 children


GridPosition(GridLayout[2, 1] (7 children), GridLayoutBase.Span(2:2, 1:1), GridLayoutBase.Inner())

GridPosition(GridLayout[2, 1] (7 children), GridLayoutBase.Span(2:2, 1:1), GridLayoutBase.Inner())